#1 - Acquiring the Data
==================


I chose to acquire my data from the <a href="https://www.svivaaqm.net/">Israeli Ministry of Environmental Protection</a>.
However, due to the site being dynamic, scraping the data was more complex. Therefore, I had to use Selenium and write a script that will imitate human interaction with the various menues through a testing Chrome driver. The script is designed to generate a report, and then scrape the data from the table into a dataframe.


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd

`Obtaining pollution data:`
-------------------------------------

The **set_driver_and_navigate** function sets the Chrome driver and navigates through the menu, returning the driver object. The **from_date** and **to_date** parameters are both used to choose the range of the data.


In [2]:
def set_driver_and_navigate(from_date, to_date):    
    
    #Set up the Chrome driver
    ser = Service("D:\\Program Files\\ChromeDriver\\chromedriver.exe")    
    op = webdriver.ChromeOptions() 
    op.add_argument("--start-maximized")
    op.add_experimental_option("detach", True)
    driver = webdriver.Chrome(service=ser, options=op)
    driver.get("https://www.svivaaqm.net/")    

    #Navigate through the menu to open the report window
    driver.execute_script("document.getElementById('menuContainerWrapper').style.display='block';")
    driver.find_element(By.XPATH, '/html/body/header/div/div[1]/div/div/li[2]/span').click()
    driver.find_element(By.XPATH, '/html/body/header/div/div[1]/div/div/li[2]/ul/li[2]/span').click()
    time.sleep(5) #allow window to load

    #Change the various options to get the relevant data
    driver.find_element(By.XPATH, '//*[@id="selectReportPeriod"]/li[6]').click()
    driver.find_element(By.XPATH, '//*[@id="fromDateDatePicker"]').clear()
    driver.find_element(By.XPATH, '//*[@id="fromDateDatePicker"]').send_keys(from_date)
    driver.find_element(By.XPATH, '//*[@id="toDateDatePicker"]').clear()
    driver.find_element(By.XPATH, '//*[@id="toDateDatePicker"]').send_keys(to_date)
    driver.find_element(By.XPATH, '//*[@id="toTimeTimePicker"]').clear()
    driver.find_element(By.XPATH, '//*[@id="toTimeTimePicker"]').send_keys(to_time) 
    
    return driver

The **generate_reports_and_write_data** function completes the menu selection by iterating through the stations in **stations_list**, locating them in the list, and generating a report for each one individually (that is due to the fact that various stations measure a different number of particles).

After the function completes scraping the data, it writes the obtained dataframe into a csv file, named **-filename-.csv**.

In [3]:
def generate_reports_and_write_data(driver, to_time, stations_list, data, filename):    
    
    stations = driver.find_elements(By.XPATH, '//*[@id="stationsBox"]/ul/li')
    c = 0 #Current item indicator    
    
    #Check the relevant stations in the treeview    
    for i in range (len(stations)):
        station_name = stations[i].find_element(By.CLASS_NAME, 'k-in').get_attribute('textContent')        
        if stations_list[c][0] == station_name:            
            path = '//*[@id="stationsBox"]/ul/li[{0}]/div/span[2]/span'.format(i+1)
            checkbox = driver.find_element(By.XPATH, path).click()
            time.sleep(1)
            
            if c == 0:
                #Change the base time to show daily average            
                combobox = driver.find_element(By.XPATH, '//*[@id="toTimeBaseCombobox"]/span/span/span[2]').click()    
                driver.find_element(By.XPATH, '//*[@id="toTimeBaseCombobox"]/span/span/span[2]').send_keys(13)    
                driver.find_element(By.XPATH, '/html/body/div[16]/div/div[2]/ul/li[13]').click()
                
            #Generate report
            time.sleep(5)
            driver.find_element(By.XPATH, '//*[@id="showResultsBtn"]').click()
            time.sleep(5)
            print("Scraping data for station {0}:".format(stations_list[c][1]))
            get_pollution_data(driver, data, stations_list[c][1])
            print("Scraping data for station {0} ended successfully.".format(stations_list[c][1]))
            #Close table window and uncheck the current station
            driver.find_element(By.XPATH, '//*[@id="report_result"]/div[1]/div[2]/img[4]').click()
            checkbox = driver.find_element(By.XPATH, path).click()
            c = c + 1
        
        if c == len(stations_list):
            break    
    
    #Finallize
    print("Data scraped successfully.")
    driver.close()    
    df = pd.DataFrame.from_dict(data)
    print("Dataframe created successfully.")
    df.to_csv('{0}.csv'.format(filename))
    print("{0}.csv created successfully".format(filename, index=False))
    return df

The function **get_pollution_data** deals with scraping the data of a particular report table, one of a given station (**station_name**). It iterates through the pages and scrapes the relevant data into a dictionary of lists (**data**, if the column is a key in the dictionary), which will later be converted into a dataframe once all the reports are scraped.

In [4]:
def get_pollution_data(driver, data, station_name):  
    
    #define needed values    
    next_page = driver.find_element(By.XPATH, '//*[@id="detailedReportGrid"]/div[4]/a[3]')   
    last_page = driver.find_element(By.XPATH, '//*[@id="detailedReportGrid"]/div[4]/a[4]')
    page_count = int(last_page.get_attribute('data-page'))
    current_page = 1          #page counter    
    
    #iterate through pages
    while current_page <= page_count:
        
        cols = driver.find_elements(By.XPATH, '//*[@id="detailedReportGrid"]/div[1]/div[2]/table/thead/tr[2]/th')
        rows = driver.find_elements(By.XPATH, '//*[@id="detailedReportGrid"]/div[2]/table/tbody[1]/tr')
        col_num = len(cols)
        row_num = len(rows)
        print("Scraping page {0} of {1}, {2} columns, {3} rows:".format(current_page, page_count, col_num, row_num))    
        
        #append data
        for i in range(row_num):
            date = rows[i].get_attribute('textContent')
            data["date"].append(date)
            data["station"].append(station_name)
            print(date)
            for j in range(col_num):
                particle_name = cols[j].get_attribute('textContent')
                if particle_name in data:
                    path = '//*[@id="detailedReportGrid"]/div[3]/table/tbody/tr[{0}]/td[{1}]'.format(i+1, j+1)
                    cell = driver.find_element(By.XPATH, path)
                    data[particle_name].append(cell.text)
                    print("{0}: {1}".format(particle_name, cell.text))
                
        #navigate to the next page
        if current_page == page_count:
            break
        time.sleep(3)
        next_page.click()
        current_page = current_page+1        
        time.sleep(5)

For my research, I chose the daily pollution data of thirteen towns across Israel, scattered over various geographic regions, along the period of five years (2016-2020). I focused on areas notorious for their high levels of pollution:

**Haifa**, a major seaport, its bay area was already designated as Israel's most polluted area.

**Hadera**, is the location of the largest power plant in the country.

**Tel Aviv**, the heart of Israel's largest metropolitan area, suffers mainly from pollution made by vehicles.

**Ariel**, located in the Samaria region, where illegal waste incineration is a common problem.

**Jerusalem**, once considered a city with good air quality, Jerusalem now suffers from a large percentage of particles.

**Beer Sheva**, is close to the Ramat Hovav industrial area and suffers from particles carried with desert dust.

**Ashdod**, is home to various polluting factories that emit dangerous particles.

In addition, I added a few locations from remote areas, for a possible comparison.

In [10]:
#Defining relevant values
from_date = "01/01/2016"
to_date = "01/01/2021"
to_time = "00:00"
stations_list = [('אריאל, אריאל', "Ariel"), 
                 ('אשדוד, ניידת -  2',"Ashdod"), 
                 ('באר שבע, מחנה נתן',"Beer Sheva"), 
                 ('חדרה, אזור התעשייה',"Hadera"),                 
                 ('חיפה, עצמאות חיפה',"Haifa"),
                 ('ירושלים, בקעה',"Jerusalem"),
                 ('כפר מנחם, כפר מנחם',"Kfar Menachem"),
                 ('כרמיאל, גליל מערבי',"Karmiel"),
                 ('עפולה, עפולה',"Afula"),
                 ('ערד, נגב מזרחי',"Arad"),
                 ('קריית גת, קריית גת',"Kiryat Gat"),
                 ('ראשון לציון, רחוב הרצל',"Rishon LeZion"),
                 ('תל אביב-יפו, לב תל אביב',"Tel Aviv")]              

data = {"date":[],"station":[],"NO":[],"NO2":[],"NOX":[]}
filename = "pollution_data_raw"

#generate the report using Chromedriver
driver = set_driver_and_navigate(from_date, to_date)
generate_reports_and_write_data(driver, to_time, stations_list, data, filename)




Scraping data for station Ariel:
Scraping page 1 of 77, 5 columns, 24 rows:
24:00 01/01/2016
NO: 1.5
NO2: 3.8
NOX: 6.1
24:00 02/01/2016
NO: 1.1
NO2: 4.9
NOX: 6.7
24:00 03/01/2016
NO: 1.4
NO2: 10.8
NOX: 13
24:00 04/01/2016
NO: 1.8
NO2: 9.7
NOX: 12.5
24:00 05/01/2016
NO: 2.1
NO2: 11.8
NOX: 15.1
24:00 06/01/2016
NO: 2.1
NO2: 13.3
NOX: 16.5
24:00 07/01/2016
NO: 2
NO2: 12.8
NOX: 15.8
24:00 08/01/2016
NO: 1.4
NO2: 5.7
NOX: 7.7
24:00 09/01/2016
NO: 1.2
NO2: 4.6
NOX: 6.4
24:00 10/01/2016
NO: 1.7
NO2: 12.8
NOX: 15.5
24:00 11/01/2016
NO: 4.6
NO2: 31
NOX: 38
24:00 12/01/2016
NO: 1.9
NO2: 16.3
NOX: 19.2
24:00 13/01/2016
NO: 2.9
NO2: 23.4
NOX: 27.9
24:00 14/01/2016
NO: 2.7
NO2: 18.3
NOX: 22.4
24:00 15/01/2016
NO: 1.9
NO2: 7.8
NOX: 10.7
24:00 16/01/2016
NO: 1.5
NO2: 7.1
NOX: 9.3
24:00 17/01/2016
NO: 2
NO2: 11.9
NOX: 14.9
24:00 18/01/2016
NO: 2.2
NO2: 9.2
NOX: 12.6
24:00 19/01/2016
NO: Calib
NO2: Calib
NOX: Calib
24:00 20/01/2016
NO: <Samp
NO2: <Samp
NOX: <Samp
24:00 21/01/2016
NO: 1.3
NO2: 26.4
NOX:

NO: 2.3
NO2: 8.6
NOX: 12
24:00 29/06/2016
NO: 2.3
NO2: 8.4
NOX: 11.7
24:00 30/06/2016
NO: 2.4
NO2: 8.2
NOX: 11.5
24:00 01/07/2016
NO: 1.5
NO2: 12.6
NOX: 14.7
24:00 02/07/2016
NO: 2.3
NO2: 4.1
NOX: 7.3
24:00 03/07/2016
NO: 0.4
NO2: 7.9
NOX: 8.2
24:00 04/07/2016
NO: 0.3
NO2: 7.8
NOX: 8.1
24:00 05/07/2016
NO: 0.9
NO2: 7.8
NOX: 9
24:00 06/07/2016
NO: <Samp
NO2: <Samp
NOX: <Samp
24:00 07/07/2016
NO: InVld
NO2: InVld
NOX: InVld
24:00 08/07/2016
NO: InVld
NO2: InVld
NOX: InVld
24:00 09/07/2016
NO: InVld
NO2: InVld
NOX: InVld
24:00 10/07/2016
NO: InVld
NO2: InVld
NOX: InVld
Scraping page 9 of 77, 5 columns, 24 rows:
24:00 11/07/2016
NO: NoData
NO2: InVld
NOX: NoData
24:00 12/07/2016
NO: NoData
NO2: InVld
NOX: NoData
24:00 13/07/2016
NO: NoData
NO2: InVld
NOX: NoData
24:00 14/07/2016
NO: NoData
NO2: InVld
NOX: NoData
24:00 15/07/2016
NO: NoData
NO2: InVld
NOX: NoData
24:00 16/07/2016
NO: NoData
NO2: InVld
NOX: NoData
24:00 17/07/2016
NO: InVld
NO2: InVld
NOX: InVld
24:00 18/07/2016
NO: InVld
NO

NOX: Down
24:00 23/12/2016
NO: Down
NO2: Down
NOX: Down
24:00 24/12/2016
NO: Down
NO2: Down
NOX: Down
24:00 25/12/2016
NO: Down
NO2: Down
NOX: Down
Scraping page 16 of 77, 5 columns, 24 rows:
24:00 26/12/2016
NO: Down
NO2: Down
NOX: Down
24:00 27/12/2016
NO: <Samp
NO2: <Samp
NOX: <Samp
24:00 28/12/2016
NO: 0.1
NO2: 5.4
NOX: 5
24:00 29/12/2016
NO: 0.2
NO2: 10.8
NOX: 10.8
24:00 30/12/2016
NO: 0.1
NO2: 5.3
NOX: 5
24:00 31/12/2016
NO: 0.1
NO2: 5.8
NOX: 5.4
24:00 01/01/2017
NO: 0.3
NO2: 10.4
NOX: 10.7
24:00 02/01/2017
NO: 0.4
NO2: 8.6
NOX: 9.1
24:00 03/01/2017
NO: <Samp
NO2: <Samp
NOX: <Samp
24:00 04/01/2017
NO: NoData
NO2: InVld
NOX: NoData
24:00 05/01/2017
NO: NoData
NO2: InVld
NOX: NoData
24:00 06/01/2017
NO: NoData
NO2: InVld
NOX: NoData
24:00 07/01/2017
NO: NoData
NO2: InVld
NOX: NoData
24:00 08/01/2017
NO: InVld
NO2: InVld
NOX: InVld
24:00 09/01/2017
NO: <Samp
NO2: <Samp
NOX: <Samp
24:00 10/01/2017
NO: 0
NO2: 14
NOX: 12.4
24:00 11/01/2017
NO: 0.2
NO2: 11.4
NOX: 9.9
24:00 12/01/2017
NO

NO: 0
NO2: 10.3
NOX: 10.3
24:00 20/06/2017
NO: 0.2
NO2: 5.8
NOX: 6.2
24:00 21/06/2017
NO: 0
NO2: 6.2
NOX: 6.3
24:00 22/06/2017
NO: -0.1
NO2: 7.6
NOX: 7.5
24:00 23/06/2017
NO: NoData
NO2: NoData
NOX: NoData
24:00 24/06/2017
NO: NoData
NO2: NoData
NOX: NoData
24:00 25/06/2017
NO: <Samp
NO2: <Samp
NOX: <Samp
24:00 26/06/2017
NO: 0.5
NO2: 16.1
NOX: 16.9
24:00 27/06/2017
NO: 0.4
NO2: 9.1
NOX: 9.6
24:00 28/06/2017
NO: 4
NO2: 22.5
NOX: 28.6
24:00 29/06/2017
NO: 0.4
NO2: 8.5
NOX: 9.1
24:00 30/06/2017
NO: 0.3
NO2: 5.5
NOX: 6
24:00 01/07/2017
NO: <Samp
NO2: <Samp
NOX: <Samp
24:00 02/07/2017
NO: <Samp
NO2: <Samp
NOX: <Samp
24:00 03/07/2017
NO: NoData
NO2: InVld
NOX: NoData
24:00 04/07/2017
NO: NoData
NO2: InVld
NOX: NoData
24:00 05/07/2017
NO: NoData
NO2: InVld
NOX: NoData
Scraping page 24 of 77, 5 columns, 24 rows:
24:00 06/07/2017
NO: <Samp
NO2: <Samp
NOX: <Samp
24:00 07/07/2017
NO: 0.4
NO2: 4.1
NOX: 4.6
24:00 08/07/2017
NO: 0.2
NO2: 4.2
NOX: 4.5
24:00 09/07/2017
NO: 0.6
NO2: 11.4
NOX: 12.4
24:

NOX: 7.2
24:00 16/12/2017
NO: 0.1
NO2: 5.3
NOX: 5.4
24:00 17/12/2017
NO: 0.2
NO2: 8.7
NOX: 9
24:00 18/12/2017
NO: 0.6
NO2: 14
NOX: 14.9
24:00 19/12/2017
NO: 1.8
NO2: 21.7
NOX: 24.3
24:00 20/12/2017
NO: 1
NO2: 16.6
NOX: 18.1
Scraping page 31 of 77, 5 columns, 24 rows:
24:00 21/12/2017
NO: 1.3
NO2: 16.6
NOX: 18.7
24:00 22/12/2017
NO: 0.6
NO2: 10.4
NOX: 11.4
24:00 23/12/2017
NO: 0.3
NO2: 6
NOX: 6.5
24:00 24/12/2017
NO: 0.7
NO2: 5.9
NOX: 6.9
24:00 25/12/2017
NO: 0.7
NO2: 8.2
NOX: 9.2
24:00 26/12/2017
NO: 0.4
NO2: 7.9
NOX: 8.5
24:00 27/12/2017
NO: 0
NO2: 4.8
NOX: 4.9
24:00 28/12/2017
NO: 1.6
NO2: 20.1
NOX: 22.6
24:00 29/12/2017
NO: 1.5
NO2: 19.8
NOX: 22.2
24:00 30/12/2017
NO: 0.1
NO2: 7.5
NOX: 7.7
24:00 31/12/2017
NO: 0.4
NO2: 8.4
NOX: 9
24:00 01/01/2018
NO: 0.3
NO2: 6.8
NOX: 7.3
24:00 02/01/2018
NO: 0.6
NO2: 7.3
NOX: 8.1
24:00 03/01/2018
NO: 0.4
NO2: 10.9
NOX: 11.4
24:00 04/01/2018
NO: 0.4
NO2: 6.3
NOX: 6.9
24:00 05/01/2018
NO: 0
NO2: 1.7
NOX: 1.8
24:00 06/01/2018
NO: 0.1
NO2: 3.8
NOX: 4
2

...

NO: 702
NO2: 151.8
NOX: 1224.9
24:00 05/06/2019
NO: 716.2
NO2: 157.2
NOX: 1251.9
24:00 06/06/2019
NO: 619
NO2: 133.6
NOX: 1079.7
24:00 07/06/2019
NO: 0.7
NO2: 17.7
NOX: 18.7
24:00 08/06/2019
NO: 1
NO2: 15.8
NOX: 17.3
24:00 09/06/2019
NO: 77.3
NO2: 35
NOX: 153.1
24:00 10/06/2019
NO: 940.1
NO2: 186.9
NOX: 1623.8
24:00 11/06/2019
NO: 748.8
NO2: 176.5
NOX: 1321
24:00 12/06/2019
NO: 660.1
NO2: 149.1
NOX: 1158
24:00 13/06/2019
NO: 741.6
NO2: 139.6
NOX: 1273
24:00 14/06/2019
NO: 112.9
NO2: 84.9
NOX: 257.4
24:00 15/06/2019
NO: 96.9
NO2: 53.9
NOX: 202.1
24:00 16/06/2019
NO: 739.4
NO2: 158.4
NOX: 1288.4
24:00 17/06/2019
NO: 611.6
NO2: 128.7
NOX: 1063.4
24:00 18/06/2019
NO: 667.4
NO2: 129.9
NOX: 1150
24:00 19/06/2019
NO: 730.4
NO2: 143.2
NOX: 1259.6
24:00 20/06/2019
NO: 746.8
NO2: 146.6
NOX: 1288.1
24:00 21/06/2019
NO: 184.2
NO2: 99.6
NOX: 381.2
24:00 22/06/2019
NO: 84.2
NO2: 42.7
NOX: 171.3
24:00 23/06/2019
NO: 797
NO2: 150
NOX: 1368.1
24:00 24/06/2019
NO: 703
NO2: 145.9
NOX: 1220.3
24:00 25/06/

NOX: 189.5
Scraping page 60 of 77, 6 columns, 24 rows:
24:00 17/11/2019
NO: 449.1
NO2: 137.6
NOX: 824.1
24:00 18/11/2019
NO: 460.8
NO2: 139.6
NOX: 843.9
24:00 19/11/2019
NO: 491.7
NO2: 150.2
NOX: 901.7
24:00 20/11/2019
NO: 492.8
NO2: 151
NOX: 904.2
24:00 21/11/2019
NO: 517.4
NO2: 165.6
NOX: 956.4
24:00 22/11/2019
NO: 35.6
NO2: 87.4
NOX: 141.8
24:00 23/11/2019
NO: 62.3
NO2: 75.8
NOX: 171
24:00 24/11/2019
NO: 446.8
NO2: 164.9
NOX: 847.9
24:00 25/11/2019
NO: 456.9
NO2: 157.1
NOX: 855.4
24:00 26/11/2019
NO: 438.7
NO2: 139.3
NOX: 809.8
24:00 27/11/2019
NO: 389.9
NO2: 137.1
NOX: 733
24:00 28/11/2019
NO: 355.1
NO2: 139
NOX: 681.9
24:00 29/11/2019
NO: 36.3
NO2: 70.7
NOX: 126.1
24:00 30/11/2019
NO: 56
NO2: 51.9
NOX: 137.6
24:00 01/12/2019
NO: 382.5
NO2: 110.5
NOX: 695.1
24:00 02/12/2019
NO: <Samp
NO2: <Samp
NOX: <Samp
24:00 03/12/2019
NO: 283.3
NO2: 113.5
NOX: 546.5
24:00 04/12/2019
NO: 288.5
NO2: 104.8
NOX: 545.7
24:00 05/12/2019
NO: 324.6
NO2: 111.2
NOX: 607.3
24:00 06/12/2019
NO: 7.4
NO2: 43

NO2: NoData
NOX: NoData
24:00 01/05/2020
NO: NoData
NO2: NoData
NOX: NoData
24:00 02/05/2020
NO: NoData
NO2: NoData
NOX: NoData
Scraping page 67 of 77, 6 columns, 24 rows:
24:00 03/05/2020
NO: NoData
NO2: NoData
NOX: NoData
24:00 04/05/2020
NO: NoData
NO2: NoData
NOX: NoData
24:00 05/05/2020
NO: NoData
NO2: NoData
NOX: NoData
24:00 06/05/2020
NO: NoData
NO2: NoData
NOX: NoData
24:00 07/05/2020
NO: NoData
NO2: NoData
NOX: NoData
24:00 08/05/2020
NO: NoData
NO2: NoData
NOX: NoData
24:00 09/05/2020
NO: NoData
NO2: NoData
NOX: NoData
24:00 10/05/2020
NO: NoData
NO2: NoData
NOX: NoData
24:00 11/05/2020
NO: <Samp
NO2: <Samp
NOX: <Samp
24:00 12/05/2020
NO: 11.4
NO2: 39.8
NOX: 57.2
24:00 13/05/2020
NO: 25.2
NO2: 56.4
NOX: 94.9
24:00 14/05/2020
NO: 20.8
NO2: 54.3
NOX: 86.1
24:00 15/05/2020
NO: 12
NO2: 44.3
NOX: 62.6
24:00 16/05/2020
NO: 4.2
NO2: 41.2
NOX: 47.6
24:00 17/05/2020
NO: 18.9
NO2: 66.6
NOX: 95.6
24:00 18/05/2020
NO: 18.9
NO2: 77.1
NOX: 106
24:00 19/05/2020
NO: 29.9
NO2: 72.1
NOX: 117.

NO2: 24.1
NOX: 29.9
24:00 17/10/2020
NO: 0
NO2: 9.3
NOX: 9.3
Scraping page 74 of 77, 6 columns, 24 rows:
24:00 18/10/2020
NO: 435.9
NO2: 145.4
NOX: 811.6
24:00 19/10/2020
NO: 416.7
NO2: 173.5
NOX: 810.5
24:00 20/10/2020
NO: 452.1
NO2: 138.3
NOX: 829.4
24:00 21/10/2020
NO: 397.1
NO2: 125.4
NOX: 732.3
24:00 22/10/2020
NO: 344.7
NO2: 130.6
NOX: 657.6
24:00 23/10/2020
NO: 4.9
NO2: 39.2
NOX: 46.7
24:00 24/10/2020
NO: 0.7
NO2: 25.1
NOX: 26.2
24:00 25/10/2020
NO: 287.9
NO2: 111.8
NOX: 551.8
24:00 26/10/2020
NO: 302.7
NO2: 116.3
NOX: 578.9
24:00 27/10/2020
NO: 254.9
NO2: 110.2
NOX: 499.8
24:00 28/10/2020
NO: 271.3
NO2: 116
NOX: 530.7
24:00 29/10/2020
NO: 272.1
NO2: 120.4
NOX: 536.4
24:00 30/10/2020
NO: 6.6
NO2: 29.8
NOX: 39.9
24:00 31/10/2020
NO: 0.6
NO2: 13.3
NOX: 14.3
24:00 01/11/2020
NO: 281.2
NO2: 112
NOX: 541.7
24:00 02/11/2020
NO: 289.3
NO2: 120.5
NOX: 562.6
24:00 03/11/2020
NO: 291.1
NO2: 118.2
NOX: 563.2
24:00 04/11/2020
NO: 277.5
NO2: 105.1
NOX: 529.2
24:00 05/11/2020
NO: 288.7
NO2: 1

,date,station,NO,NO2,NOX
0,24:00 01/01/2016,Ariel,1.5,3.8,6.1
1,24:00 02/01/2016,Ariel,1.1,4.9,6.7
2,24:00 03/01/2016,Ariel,1.4,10.8,13
3,24:00 04/01/2016,Ariel,1.8,9.7,12.5
4,24:00 05/01/2016,Ariel,2.1,11.8,15.1
...,...,...,...,...,...
23746,24:00 27/12/2020,Tel Aviv,336.4,123.1,637.3
23747,24:00 28/12/2020,Tel Aviv,295.4,107.1,558.6
23748,24:00 29/12/2020,Tel Aviv,337.9,118.7,635.2
23749,24:00 30/12/2020,Tel Aviv,313.1,111.3,589.9


`Obtaining precipitation data:`
-------------------------------------

I manually downloaded a csv file containing the precipitation data for the thirteen locations in my research. The station names do not match with the ones from the pollution data, and some data had to be taken from different towns that are adjacent to the ones in my research.

In [5]:
def load_csv(filename):
    return pd.read_csv(filename, encoding = "ISO-8859-8")    

In [6]:
filename = 'precipitation_data_raw.csv'
df = load_csv(filename)
df

,שם תחנה,מספר תחנה,תאריך,"כמות גשם יומית(מ""מ)",קוד גשם יומי()
0,עפולה ניר העמק ...,221604,01/01/2016,10.3,0
1,עפולה ניר העמק ...,221604,02/01/2016,8.3,0
2,עפולה ניר העמק ...,221604,03/01/2016,0.9,0
3,עפולה ניר העמק ...,221604,07/01/2016,29.5,0
4,עפולה ניר העמק ...,221604,08/01/2016,39.9,0
...,...,...,...,...,...
3729,תל-אביב חוף ...,136320,15/12/2020,19.9,0
3730,תל-אביב חוף ...,136320,16/12/2020,58.2,0
3731,תל-אביב חוף ...,136320,17/12/2020,8.5,0
3732,תל-אביב חוף ...,136320,23/12/2020,5.3,0
